In [2]:
import tensorflow as tf
import numpy as np

In [3]:
#Verify Tensorflow is using GPU
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
lst = []

with open('../data/full_tab_string.txt') as small_pf:

    tmp_list = []
    for line in small_pf:
        line = line.rstrip("\n")
        if line == "":
            lst.append(tmp_list)
            tmp_list = []
        else:
            tmp_list.extend(line.split())

    if tmp_list:  # add last one
        lst.append(tmp_list)
lst = lst[0]

In [5]:
text = ""
with open('../data/full_tab_string.txt') as small_pf:
    text = small_pf.read()

In [6]:
#Can't use set because tab encodings are not hashable types
def unique(list1):
    unique_set = set()
    for x in list1:
        if x not in unique_set:
            unique_set.add(x)
    return unique_set

In [7]:
#list of all tab locations
tab_vocab = unique(lst)

In [8]:
#this is a quick fix and needs to be done at the raw data level
new_list = []
for tab in tab_vocab:
    if int(tab[1:]) <= 22:
        new_list.append(tab)
tab_vocab = new_list

In [9]:
ids_from_chars = tf.keras.layers.StringLookup(
    num_oov_indices = 1, 
    vocabulary=list(tab_vocab), 
    mask_token=None)
print(ids_from_chars.get_vocabulary())

['[UNK]', 'B8', 'E2', 'e21', 'A6', 'D21', 'D14', 'E17', 'E1', 'A9', 'A18', 'D5', 'G13', 'A2', 'D18', 'B6', 'B3', 'A12', 'B12', 'E7', 'A5', 'G7', 'A19', 'D19', 'G20', 'G17', 'E15', 'e18', 'D17', 'G16', 'A10', 'D7', 'A16', 'e6', 'B16', 'G21', 'G8', 'e7', 'B21', 'D10', 'e0', 'E13', 'A8', 'A15', 'e12', 'G3', 'E0', 'D3', 'G15', 'D11', 'e4', 'e1', 'G22', 'D9', 'E19', 'A22', 'G6', 'e10', 'D20', 'E20', 'B14', 'A11', 'A21', 'B20', 'E16', 'e19', 'A4', 'E8', 'B13', 'D13', 'G12', 'B22', 'A0', 'D1', 'B2', 'D0', 'B18', 'E9', 'E10', 'E5', 'e15', 'e3', 'B9', 'B7', 'A7', 'A1', 'G5', 'e5', 'G4', 'D8', 'e9', 'e8', 'D16', 'G14', 'e11', 'e14', 'B19', 'E18', 'e17', 'E12', 'e16', 'A17', 'e20', 'G0', 'D12', 'G1', 'D2', 'e13', 'D22', 'e22', 'D15', 'G19', 'G2', 'A20', 'A3', 'G18', 'G10', 'G11', 'D4', 'B17', 'D6', 'B5', 'B10', 'E4', 'B11', 'B4', 'E6', 'A14', 'E14', 'e2', 'E3', 'B0', 'B1', 'B15', 'G9', 'E11', 'A13', 'E22']


In [10]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)


In [11]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids) + " ", axis=-1)

In [12]:
all_ids = ids_from_chars(tf.strings.split(text))

In [13]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)
type(ids_dataset)

tensorflow.python.data.ops.dataset_ops.TensorSliceDataset

In [14]:
seq_length = 10

In [15]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)
print(type(sequences))
for seq in sequences.take(1):
  print(chars_from_ids(seq))
  print(type(seq))

<class 'tensorflow.python.data.ops.dataset_ops.BatchDataset'>
tf.Tensor([b'D7' b'G5' b'B5' b'e5' b'e7' b'D6' b'B5' b'G5' b'e7' b'e8' b'D5'], shape=(11,), dtype=string)
<class 'tensorflow.python.framework.ops.EagerTensor'>


In [16]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [17]:
dataset = sequences.map(split_input_target)
dataset



<MapDataset element_spec=(TensorSpec(shape=(10,), dtype=tf.int64, name=None), TensorSpec(shape=(10,), dtype=tf.int64, name=None))>

In [18]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'D7 G5 B5 e5 e7 D6 B5 G5 e7 e8 '
Target: b'G5 B5 e5 e7 D6 B5 G5 e7 e8 D5 '


In [19]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset element_spec=(TensorSpec(shape=(64, 10), dtype=tf.int64, name=None), TensorSpec(shape=(64, 10), dtype=tf.int64, name=None))>

In [20]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 64

# Number of RNN units
rnn_units = 1024

In [21]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)

    #adjust the dense units size                         
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [22]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [23]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 10, 138) # (batch_size, sequence_length, vocab_size)


In [24]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  8832      
                                                                 
 gru (GRU)                   multiple                  3348480   
                                                                 
 dense (Dense)               multiple                  141450    
                                                                 
Total params: 3,498,762
Trainable params: 3,498,762
Non-trainable params: 0
_________________________________________________________________


In [25]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [26]:
sampled_indices

array([  5,  93,  27, 110,  54, 112, 131,  58,   4, 133], dtype=int64)

In [27]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [28]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 10, 138)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.927367, shape=(), dtype=float32)


In [29]:
model.compile(optimizer='adam', loss=loss)

In [30]:
import os
# Directory where the checkpoints will be saved
checkpoint_dir = './tab_training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [55]:
EPOCHS = 1000

In [56]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/1000
411/411 [==============================] - 3s 6ms/step - loss: 0.6785
Epoch 2/1000
411/411 [==============================] - 3s 6ms/step - loss: 0.6810
Epoch 3/1000
411/411 [==============================] - 3s 6ms/step - loss: 0.6816
Epoch 4/1000
411/411 [==============================] - 3s 6ms/step - loss: 0.6839
Epoch 5/1000
411/411 [==============================] - 3s 6ms/step - loss: 0.6857
Epoch 6/1000
411/411 [==============================] - 3s 6ms/step - loss: 0.6872
Epoch 7/1000
411/411 [==============================] - 3s 6ms/step - loss: 0.6855
Epoch 8/1000
411/411 [==============================] - 3s 6ms/step - loss: 0.6819
Epoch 9/1000
411/411 [==============================] - 3s 6ms/step - loss: 0.6780
Epoch 10/1000
411/411 [==============================] - 3s 6ms/step - loss: 0.6776
Epoch 11/1000
411/411 [==============================] - 3s 6ms/step - loss: 0.6788
Epoch 12/1000
411/411 [==============================] - 3s 6ms/step - loss: 0.6816
E

KeyboardInterrupt: 

In [42]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, omit, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.

    skip_ids = self.ids_from_chars(omit)[:, None]

    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, omit, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]

    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.

    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [43]:
from MidiToTabOptions import *
tab_midi_dict = generate_tab_midi_pairs()

In [47]:
from MidiToArray import *
notes = MidiToArray(r"C:\Users\alecr\desktop\mutab\Sandbox\samples\sample.mid")
notes = notes[1:12]
notes



[60, 64, 69, 71, 64, 60, 71, 72, 64, 60, 72]

In [53]:
#Array of Midi notes
#notes = [60, 61, 63, 60, 56, 58]
states = None

#First tab note, hard coded
base_tab = 'D7 '

#Dynamically starting input 
append_tab = tf.constant([base_tab])
for note in notes:

    #Finds all tab options for the current note and 
    #removes them from the omitted option list in the OneStep function
    options = ['[UNK]']
    options.extend(tab_midi_dict[note])
    omit = list(set(ids_from_chars.get_vocabulary()) - set(options))
    omit.insert(0, '[UNK]')

    omit_sorted = []
    for item in ids_from_chars.get_vocabulary():
        if item in omit:
            omit_sorted.append(item)

    
    #generates the next predicted tab note out of the set of possible options   
    one_step_model = OneStep(model, chars_from_ids, ids_from_chars, omit_sorted)

    #bootstrap the predictions 
    bootstrap_list = []    
    for i in range(1):
        next_char, states = one_step_model.generate_one_step(append_tab, omit_sorted, states=states)    
        bootstrap_list.append(next_char.numpy()[0].decode('utf-8'))
    next_tab = max(set(bootstrap_list), key=bootstrap_list.count)


    #append new tab to input list
    append_tab = tf.constant([append_tab.numpy()[0].decode('utf-8') + next_tab + ' '])
tf.keras.backend.clear_session()

In [54]:
string = str(append_tab.numpy()[0].decode('utf-8'))
array = string.split(' ')[:-1]
print(convert_array_to_tab(array))

e|------0-----------------------------
B|------------------------13-------13-
G|---5-----14-16-9--5--16----9--5-----
D|7-----------------------------------
A|------------------------------------
E|------------------------------------



In [39]:
print(append_tab)

tf.Tensor([b'D7 G2 G5 G9 B10 e7 G9 G5 e7 B13 e0 G5 B13 '], shape=(1,), dtype=string)
